In [ ]:
import time
import datetime
import praw
import pandas as pd
import unicodedata
from sqlalchemy import create_engine
from sqlalchemy.types import String

In [ ]:
STARTSEARCH = '2015.04.01 00:00:00'
ENDSEARCH = '2015.04.30 23:59:59'
POSTPERIOD = 'Apr2015'
TOTABLE = 'posts1000_2'

USERAGENT = "/u/insight_fu comments"
SUBREDDIT = "girlgamers"
MAXPOSTS = 1000

In [ ]:
r = praw.Reddit(USERAGENT)

In [ ]:
pattern = '%Y.%m.%d %H:%M:%S'
startEpochSearch = time.mktime(time.strptime(STARTSEARCH, pattern))
endEpochSearch = time.mktime(time.strptime(ENDSEARCH, pattern))
searchTerm = "timestamp:" + str(startEpochSearch)[:-2] + ".." + str(endEpochSearch)[:-2]

In [ ]:
subreddit = r.get_subreddit(SUBREDDIT)
searchSubreddit = subreddit.search(searchTerm, sort = "top", syntax = "cloudsearch", limit = MAXPOSTS)
posts = list(searchSubreddit)

In [ ]:
s = []
for post in posts:
    s.append({'subreddit': post.subreddit,
              'postPeriod': POSTPERIOD,
              'postID': post.id,
              'postCreated': post.created,
              'postTitle': post.title,
              'postLink': post.permalink,
              'postAuthor': post.author,
              'postNumComments': post.num_comments,
              'postScore': post.score,
              'postBody': post.selftext if post.is_self is True else post.url})

In [ ]:
sDF = pd.DataFrame(s)

In [ ]:
convertThese = ['postAuthor', 'subreddit', 'postCreated']
for convert in convertThese:
    sDF[convert] = sDF[convert].map(str)

In [ ]:
convertThese = ['postBody', 'postID', 'postLink', 'postTitle']
for convert in convertThese:
    sDF[convert] = sDF[convert].map(lambda x:unicodedata.normalize('NFKD', x).encode('ascii', 'ignore'))

In [ ]:
engine = create_engine("mysql+pymysql://root@localhost/" + str(SUBREDDIT))
sDF.to_sql(TOTABLE, engine, if_exists="append")